In [ ]:
import zipfile
import requests
import cv2
import matplotlib.pyplot as plt
import glob 
import random
import os
import torch
from ultralytics import YOLO

In [ ]:
# Function to convert bounding boxes in YOLO format to xmin, ymin, xmax, ymax.
def yolo2bbox(bboxes):
    xmin, ymin = bboxes[0]-bboxes[2]/2, bboxes[1]-bboxes[3]/2
    xmax, ymax = bboxes[0]+bboxes[2]/2, bboxes[1]+bboxes[3]/2
    return xmin, ymin, xmax, ymax

In [ ]:
def plot_box(image, bboxes, labels):
    # Need the image height and width to denormalize
    # the bounding box coordinates
    h, w, _ = image.shape
    for box_num, box in enumerate(bboxes):
        x1, y1, x2, y2 = yolo2bbox(box)
        # Denormalize the coordinates.
        xmin = int(x1*w)
        ymin = int(y1*h)
        xmax = int(x2*w)
        ymax = int(y2*h)

        thickness = max(2, int(w/275))
                
        cv2.rectangle(
            image, 
            (xmin, ymin), (xmax, ymax),
            color=(0, 0, 255),
            thickness=thickness
        )
    return image

In [ ]:
# Function to plot images with the bounding boxes.
def plot(image_paths, label_paths, num_samples):
    all_images = []
    all_images.extend(glob.glob(image_paths+'/*.jpg'))
    all_images.extend(glob.glob(image_paths+'/*.JPG'))
    
    all_images.sort()

    num_images = len(all_images)
    
    plt.figure(figsize=(15, 12))
    for i in range(num_samples):
        j = random.randint(0,num_images-1)
        image_name = all_images[j]
        image_name = '.'.join(image_name.split(os.path.sep)[-1].split('.')[:-1])
        image = cv2.imread(all_images[j])
        with open(os.path.join(label_paths, image_name+'.txt'), 'r') as f:
            bboxes = []
            labels = []
            label_lines = f.readlines()
            for label_line in label_lines:
                label = label_line[0]
                bbox_string = label_line[2:]
                x_c, y_c, w, h = bbox_string.split(' ')
                x_c = float(x_c)
                y_c = float(y_c)
                w = float(w)
                h = float(h)
                bboxes.append([x_c, y_c, w, h])
                labels.append(label)
        result_image = plot_box(image, bboxes, labels)
        plt.subplot(2, 2, i+1)
        plt.imshow(result_image[:, :, ::-1])
        plt.axis('off')
    plt.subplots_adjust(wspace=1)
    plt.tight_layout()
    plt.show()

In [ ]:
import os
os.listdir('datasets/elderly/train/images/')

In [ ]:
# Visualize a few training images.
plot(
    image_paths='datasets/elderly/train/images/', 
    label_paths='datasets/elderly/train/labels/',
    num_samples=4,
)

Create Yaml file

In [ ]:
%%writefile fall_v8.yaml
path: 'D:/Python_Env/HumanFallDetection/final_fall/datasets/elderly/'
train: 'D:/Python_Env/HumanFallDetection/final_fall/datasets/elderly/train/'
val: 'D:/Python_Env/HumanFallDetection/final_fall/datasets/elderly/valid/'
test: 'D:/Python_Env/HumanFallDetection/final_fall/datasets/elderly/test/'
# class names
names: 
  0: 'fall'
  1: 'notfallen'

Default value is 0 set 

In [ ]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '0'

# Train Model

Check is cuda is available

In [ ]:
import torch

def is_cuda_available():
    return torch.cuda.is_available()

print(is_cuda_available())

In [ ]:
from ultralytics import YOLO
from PIL import Image
from torchvision.utils import draw_bounding_boxes
import torch
from pathlib import Path
from ultralytics.utils import metrics

from ultralytics import YOLO
import os
# os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
# os.environ['TORCH_USE_CUDA_DSA']='1' 
# print(os.environ['CUDA_LAUNCH_BLOCKING'])
# print(os.environ['TORCH_USE_CUDA_DSA'])

# Load the pre-trained model
model = YOLO("yolov8n.pt")

# Fine-tune the model
model.train(
    data='fall_v8.yaml', 
    epochs=10, 
    imgsz=1280, 
    batch=8,#works with batch=8
    save_period=1,
    device=[0],
    optimizer='auto',
)

Test Model with an Image

In [ ]:
from ultralytics import YOLO
import cv2
# Create a YOLO model
model = YOLO('best.pt')

# Run the YOLO model on the input
results = model.predict(source="D:/Python_Env/HumanFallDetection/final_fall/datasets/elderly/valid/images/img_104.jpg", show=True)

# Print the results
print(results)
cv2.waitKey(0)
cv2.destroyAllWindows()

Test Model with a Video

In [ ]:
from ultralytics import YOLO
import cv2

# Create a YOLO model
model = YOLO('../final_fall/trained_models/best.pt')

# Define a video source
video_source = '../final_fall/VideoFolder/fall4.mp4'

# Create a video capture object
cap = cv2.VideoCapture(video_source)

# Check if the video capture object was created successfully
if not cap.isOpened():
    print("Error opening video file")

# Loop through the video
while cap.isOpened():
    # Capture frame-by-frame
    ret, frame = cap.read()

    # If the frame was captured successfully
    if ret:
        # Run the YOLO model on the frame
        results = model.predict(frame, show=True)

        # Print the results
        print(results)
        # Break the loop if the 'q' key is pressed
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # Break the loop if the 'q' key is pressed
    else:
        break

# Release the video capture object
cap.release()

# Destroy all windows
cv2.destroyAllWindows()

# Release the video capture object
cap.release()

# Destroy all windows
cv2.destroyAllWindows()

In [ ]:
!yolo task=detect mode=predict model=best.pt source=D:/Python_Env/HumanFallDetection/final_fall/datasets/elderly/valid/images imgsz=1280 name=yolov8n_v8_50e_infer1280

In [ ]:
# Plot and visualize images in a 2x2 grid.
def visualize(result_dir, num_samples=4):
    """
    Function accepts a list of images and plots
    them in a 2x2 grid.
    """
    plt.figure(figsize=(20, 12))
    image_names = glob.glob(os.path.join(result_dir, '*.jpg'))
    random.shuffle(image_names)
    for i, image_name in enumerate(image_names):
        image = plt.imread(image_name)
        plt.subplot(2, 2, i+1)
        plt.imshow(image)
        plt.axis('off')
        if i == num_samples-1:
            break
    plt.tight_layout()
    plt.show()

In [ ]:
visualize('D:/Python_Env/HumanFallDetection/runs/detect/yolov8n_v8_50e_infer1280')